In [14]:
import pandas as pd
import numpy as np
import math
import pickle
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd
import birch_bellwether_v2 as birch_bellwether

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [15]:
def cluster_driver(df,print_tree = False):
    X = df.apply(pd.to_numeric)
    cluster = birch.birch(branching_factor=20)
        #X.set_index('Project Name',inplace=True)
    cluster.fit(X)
    cluster_tree,max_depth = cluster.get_cluster_tree()
        #cluster_tree = cluster.model_adder(cluster_tree)
    if print_tree:
        cluster.show_clutser_tree()
    return cluster,cluster_tree,max_depth

def build_BIRCH(attr_df):
    cluster,cluster_tree,_ = cluster_driver(attr_df)
    return cluster,cluster_tree

def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.iloc[i].name
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-1.5}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n

In [16]:
#get cluster wise data for summarzation using median
def find_bellwether(data_source1,data_store,clusters,path,fold):
    df_train = pd.read_pickle(data_source1 + '/train_data.pkl')
    cluster,cluster_tree = build_BIRCH(df_train)
    cluster_ids = []
    cluster_structure = {}
    size = {}
    for key in cluster_tree:
        if cluster_tree[key].depth != None:
            cluster_ids.append(key)
            if cluster_tree[key].depth not in cluster_structure.keys():
                cluster_structure[cluster_tree[key].depth] = {}
            cluster_structure[cluster_tree[key].depth][key] = cluster_tree[key].parent_id
            size[key] = cluster_tree[key].size
    goals = ['recall','precision','pf']
    for _ in range(1):
        bell_df = {}
        score_df = pd.read_csv(data_source1 + '/bellwether_cdom_1.csv')
        score_df = score_df.rename(columns = {'Unnamed: 0':'id'})
        _cluster_bellwethers = score_df.bellwether.values.tolist()
        bell = birch_bellwether.bellwether(path,score_df)
        final_score = bell.bellwether(_cluster_bellwethers,_cluster_bellwethers)
        _results = {}
        for goal in goals:    
            for s_project in final_score.keys():
                if s_project not in _results.keys():
                    _results[s_project] = {}
                    _temp = []
                for d_projects in final_score[s_project].keys():
                    if goal == 'g':
                        _goal = 'g-score'
                    else:
                        _goal = goal
                    _score = np.median(final_score[s_project][d_projects][_goal])
                    _temp.append(np.median(final_score[s_project][d_projects][_goal]))
                if goal not in _results[s_project].keys():
                    _results[s_project][goal] = []
                _results[s_project][goal] = np.median(_temp)
        _df = pd.DataFrame.from_dict(_results, orient = 'index')
        dom_score = []
        for row_id in range(_df.shape[0]):
            project_name = _df.iloc[row_id].name
            row = _df.iloc[row_id].tolist()
            wins = dominate(_df,row,project_name,goals)
            dom_score.append(wins)
        _df['wins'] = dom_score
        print(_df)
        c_dom = _df.wins.values.tolist()
        best_project = _df.index[c_dom.index(max(c_dom))]
        best_project_perf = _df.loc[best_project].values.tolist()
        best_project_perf.append(best_project)
        bell_df[key] = best_project_perf
    perf_df = pd.DataFrame.from_dict(bell_df, orient = 'index', columns = ['recall','precision','pf','cdom','bellwether'])    
    print(data_source1 + '/bellwether_cdom_0.csv')
    perf_df.to_csv(data_source1 + '/bellwether_cdom_0.csv')
            
            
            

In [17]:
for i in range(4,20):
    print(i)
    path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/data/1385/converted'
    data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/2/fold_' + str(i)
    data_store = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/0/fold_' + str(i)
    if platform.system() == 'Darwin' or platform.system() == 'Linux':
        _dir = data_source1 + '/'
    else:
        _dir = data_source1 + '\\'

    clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]
    find_bellwether(data_source1,data_store,clusters,path,i)

4
gephex.csv
emftriple.csv
les-indemodables.csv
retromenu.csv
qse.csv
uwom-server.csv
magicwars.csv
turbotrader-bos.csv
xmemcached.csv
                      recall  precision     pf  wins
emftriple.csv           0.04      0.350  0.425     0
gephex.csv              0.52      0.445  0.430     1
les-indemodables.csv    0.43      0.415  0.400     2
magicwars.csv           0.96      0.430  0.370     8
qse.csv                 0.62      0.445  0.360     7
retromenu.csv           0.08      0.430  0.355     3
turbotrader-bos.csv     0.45      0.430  0.355     5
uwom-server.csv         0.38      0.430  0.355     4
xmemcached.csv          0.50      0.430  0.350     6
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/2/fold_4/bellwether_cdom_0.csv
5
ingex.csv
columba.csv
ng4j.csv
lamp.csv
netatalk.csv
pdfedit.csv
magicwars.csv
pentahoanalysistool.csv
xmemcached.csv
                         recall  precision     pf  wins
columba.csv                0.50 

gpsmid.csv
arcallians.csv
lamp.csv
netatalk.csv
rockbox.csv
magicwars.csv
turbotrader-bos.csv
xmemcached.csv
                     recall  precision     pf  wins
arcallians.csv         0.65      0.545  0.420     5
gpsmid.csv             0.64      0.530  0.440     1
jmoney.csv             0.30      0.505  0.450     0
lamp.csv               0.13      0.560  0.400     3
magicwars.csv          0.91      0.510  0.390     6
netatalk.csv           0.29      0.585  0.340     8
rockbox.csv            0.54      0.560  0.345     7
turbotrader-bos.csv    0.47      0.500  0.370     4
xmemcached.csv         0.51      0.500  0.390     2
/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_rf_v2/2/fold_15/bellwether_cdom_0.csv
16
dxengine.csv
emftriple.csv
kml.csv
poormans.csv
minig.csv
nassp.csv
magicwars.csv
codesmith.csv
xmemcached.csv
                recall  precision     pf  wins
codesmith.csv     0.86      0.370  0.345     4
dxengine.csv      0.13      0.360  